```{include} ../math-definitions.md
```

In [33]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import pandas_datareader as pdr # pandas datareader es una libreria que permite descargar datos financieros de diversas fuentes como Yahoo Finance, Google Finance, 

In [34]:
# Customize plotly's default templates
import plotly.io as pio
pio.templates.default = "gridon"

In [39]:
from style import plotly_apply
plotly_apply()

# El filtro de Hodrick y Prescott
**Otro método para remover una tendencia**


## Desagregación de una serie de tiempo

Tenemos una muestra de $T$ observaciones de la variable aleatoria $Y_t$:
\begin{equation}
	\{y_1, y_2,\dots, y_T \}
\end{equation}

$Y_t$ tiene dos componentes: crecimiento (tendencia) $s_t$ y ciclo $c_t$.
\begin{equation}
    {y_t}{original} = {s_t}{tendencia} + {c_t}{ciclo}
\end{equation}

Asumimos que la tendencia es una curva *suave*, aunque no necesariamente una línea recta.

```{figure} figures/hp-filter-intuition.png
```


## Objetivos en conflicto

- Partiendo de $y_t$, \cite{Hodrick-Prescott:1997} “extraen” la tendencia $s_t$
\begin{equation*}
\{s_1, s_2, \dots, s_T\},
\end{equation*}

tratando de balancear dos objetivos mutuamente excluyentes:

1. el ajuste a los datos originales, es decir, $y_t-s_t$ debe ser pequeño.
2. la tendencia resultante debe ser suaver, por lo que los cambios de pendiente $(s_{t+1}-s_t)-(s_t-s_{t-1})$ también deben ser pequeños.

La importancia relativa de estos dos factores es ponderada con el parámetro $\lambda$.


## El filtro de Hodrick y Prescott
Formalmente, la tendencia la definen por:
\begin{align*}
s_i^{HP} &=   \argmin_{s_1,\dots,s_T}\left\{\sum_{t=1}^{T}\left(y_t-s_t\right)^2 +  \lambda \sum_{t=2}^{T-1}\left[\left(s_{t+1}-s_t\right) - \left(s_t-s_{t-1}\right)  \right]^2    \right\}\\ \\
         &= \argmin_{s_1,\dots,s_T}\left\{\sum_{t=1}^{T}\left(y_t-s_t\right)^2 +  \lambda \sum_{t=2}^{T-1}\left(s_{t+1}-2s_t + s_{t-1}\right)^2 \right\}
\end{align*}


## Un truco de álgebra lineal
Definimos las matrices
\begin{equation*}
Y = \begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_T
\end{bmatrix}
\qquad
S = \begin{bmatrix}
s_1 \\
s_2 \\
\vdots \\
s_T
\end{bmatrix}
\end{equation*}

\begin{equation*}
A_{T-2\times T} =
\begin{bmatrix}
1 & -2 & 1 & 0 & \dots & 0 & 0 & 0 & 0 \\
0 & 1 & -2 & 1 & \dots & 0 & 0 & 0 & 0 \\
&  &  &  & \ddots &  &  &  &  \\
0 & 0 & 0 & 0 & \dots & 0 & 1 & -2 & 1
\end{bmatrix}
\end{equation*}

Reescribimos el problema de optimización
\begin{align*}
	s_i^{HP} &=  \argmin_{s_1,\dots,s_T}\left\{\sum_{t=1}^{T}\left(y_t-s_t\right)^2 +  \lambda \sum_{t=2}^{T-1}\left(s_{t+1}-2s_t + s_{t-1}\right)^2 \right\}\\  
           &= \argmin_{S}\left\{(Y-S)'(Y-S) +  \lambda (AS)'(AS) \right\} \\
           &= \argmin_{S}\left\{Y'Y - 2Y'S +  S'(I + \lambda A'A)S \right\}
\end{align*}


## Resolviendo el problema

Las condiciones de primer orden son
\begin{align*}
S^{HP} &= \argmin_{S}\left\{Y'Y - 2Y'S +  S'(I + \lambda A'A)S \right\} \\
\Rightarrow & - 2Y + 2\left(I + \lambda A'A\right)S = 0
\end{align*}

Por lo que el filtro HP es
\begin{align*}
S^{HP}                 &= \left(I + \lambda A'A\right)^{-1}Y \tag{tendencia} \\
C^{HP} \equiv Y-S^{HP} &= \left[I - \left(I + \lambda A'A\right)^{-1}\right]Y \tag{ciclo}
\end{align*}



{{ empieza_ejemplo }} El filtro HP {{ fin_titulo_ejemplo }}
Asumimos que tenemos $T=5$ datos $Y=\left[y_1,y_2,y_3,y_4,y_5\right]'$ y que $\lambda=4$. Los datos de tendencia $S=\left[s_1,s_2,s_3,s_4,s_5\right]'$ están dados por:

\begin{align*}
 s_1 &= & &0.67y_1 &+ & 0.36y_2 &+ & 0.13y_3 &- & 0.02y_4 &- & 0.14y_5 \\
 s_2 &= & &0.36y_1 &+ & 0.34y_2 &+ & 0.23y_3 &+ & 0.10y_4 &- & 0.02y_5 \\
 s_3 &= & &0.13y_1 &+ & 0.23y_2 &+ & 0.29y_3 &+ & 0.23y_4 &+ & 0.13y_5 \\
 s_4 &= &-&0.02y_1 &+ & 0.10y_2 &+ & 0.23y_3 &+ & 0.34y_4 &+ & 0.36y_5 \\
 s_5 &= &-&0.14y_1 &- & 0.02y_2 &+ & 0.13y_3 &+ & 0.36y_4 &+ & 0.67y_5
\end{align*}

Observe que cada dato de tendencia $s_t$ es simplemente un promedio ponderado de todos los datos en $Y$. Además. algunas de las ponderaciones son negativas!

Por otra parte, los datos del ciclo $C=\left[c_1,c_2,c_3,c_4,c_5\right]'$ están dados por:

\begin{align*}
c_1 &= & &0.33y_1 &- & 0.36y_2 &- & 0.13y_3 &+ & 0.02y_4 &+ & 0.14y_5 \\
c_2 &= &-&0.36y_1 &+ & 0.66y_2 &- & 0.23y_3 &- & 0.10y_4 &+ & 0.02y_5 \\
c_3 &= &-&0.13y_1 &- & 0.23y_2 &+ & 0.71y_3 &- & 0.23y_4 &- & 0.13y_5 \\
c_4 &= & &0.02y_1 &- & 0.10y_2 &- & 0.23y_3 &+ & 0.66y_4 &- & 0.36y_5 \\
c_5 &= & &0.14y_1 &+ & 0.02y_2 &- & 0.13y_3 &- & 0.36y_4 &+ & 0.33y_5 \\
\end{align*}

De nuevo, observe que cada dato del ciclo $c_t$ es un promedio ponderado de todos los puntos en $Y$, pero donde las ponderaciones suman cero.

{{ termina_ejemplo }}



## Escogiendo $\lambda$

El resultado del filtro es muy sensible a la escogencia de $\lambda$. Como regla habitual, $\lambda$ se escoge según la frecuencia de los datos

- Anuales $\Rightarrow 100$
- Trimestrales $\Rightarrow 1600$
- Mensuales $\Rightarrow 14400$



{{ empieza_ejemplo }} Filtered series when $\lambda=1600$ {{ fin_titulo_ejemplo }}

In [40]:
# gdp = pdr.get_data_fred('GDPC1', start=1947)   # Producto Interno Bruto Real de Estados Unidos (Billones de Dólares)
# gdp.to_csv('GDP EEUU')

In [41]:
gdp = pd.read_csv('GDP EEUU', index_col=0, parse_dates=True, na_values='.')

In [42]:
gdp

,GDPC1
DATE,
1947-01-01,2182.681
1947-04-01,2176.892
1947-07-01,2172.432
1947-10-01,2206.452
1948-01-01,2239.682
...,...
2024-04-01,23223.906
2024-07-01,23400.294
2024-10-01,23542.349


In [43]:
N = len(gdp)
N

314

In [44]:
gdp.plot(title="Producto Interno Bruto Real de Estados Unidos (Billones de Dólares)")

In [45]:
def construccion_de_A(T: int) -> np.ndarray:
    """Construye la matriz A, que es una matriz de diferencias finitas de segundo orden."""
    A = np.zeros((T-2, T))      # Crea una matriz de ceros de tamaño (T-2) x T
    for i in range(T-2):        # Llena la matriz A con los coeficientes de diferencias finitas
        A[i, i:i+3] = 1, -2, 1
    return A    

A = construccion_de_A(N)
A

array([[ 1., -2.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1., -2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -2.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1., -2.,  1.]], shape=(312, 314))

In [46]:
np.identity(N)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], shape=(314, 314))

In [47]:
A.T @ A

array([[ 1., -2.,  1., ...,  0.,  0.,  0.],
       [-2.,  5., -4., ...,  0.,  0.,  0.],
       [ 1., -4.,  6., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  6., -4.,  1.],
       [ 0.,  0.,  0., ..., -4.,  5., -2.],
       [ 0.,  0.,  0., ...,  1., -2.,  1.]], shape=(314, 314))

In [48]:
lambda_ = 1600

In [49]:
lambda_ * (A.T @ A)

array([[ 1600., -3200.,  1600., ...,     0.,     0.,     0.],
       [-3200.,  8000., -6400., ...,     0.,     0.,     0.],
       [ 1600., -6400.,  9600., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ...,  9600., -6400.,  1600.],
       [    0.,     0.,     0., ..., -6400.,  8000., -3200.],
       [    0.,     0.,     0., ...,  1600., -3200.,  1600.]],
      shape=(314, 314))

In [50]:
np.linalg.inv(np.identity(N) + lambda_ * (A.T @ A))

array([[ 2.00556217e-01,  1.78203312e-01,  1.56350059e-01, ...,
        -1.63190994e-16, -1.40494792e-16, -1.17725011e-16],
       [ 1.78203312e-01,  1.60833073e-01,  1.43351457e-01, ...,
        -1.69212043e-16, -1.54897322e-16, -1.40494792e-16],
       [ 1.56350059e-01,  1.43351457e-01,  1.30255137e-01, ...,
        -1.75131098e-16, -1.69212043e-16, -1.63190994e-16],
       ...,
       [-1.63190994e-16, -1.69212043e-16, -1.75131098e-16, ...,
         1.30255137e-01,  1.43351457e-01,  1.56350059e-01],
       [-1.40494792e-16, -1.54897322e-16, -1.69212043e-16, ...,
         1.43351457e-01,  1.60833073e-01,  1.78203312e-01],
       [-1.17725011e-16, -1.40494792e-16, -1.63190994e-16, ...,
         1.56350059e-01,  1.78203312e-01,  2.00556217e-01]],
      shape=(314, 314))

In [51]:
tendencia = np.linalg.inv(np.identity(N) + lambda_ * (A.T @ A)) @ gdp
tendencia

,GDPC1
0,2114.616270
1,2139.870604
2,2165.167478
3,2190.572572
4,2216.156104
...,...
309,23143.505005
310,23295.853596
311,23448.061010
312,23600.153234


In [52]:
gdp_copy = gdp.copy()
gdp_copy

,GDPC1
DATE,
1947-01-01,2182.681
1947-04-01,2176.892
1947-07-01,2172.432
1947-10-01,2206.452
1948-01-01,2239.682
...,...
2024-04-01,23223.906
2024-07-01,23400.294
2024-10-01,23542.349


In [53]:
# Plot the original series and the trend
fig = gdp.plot(title="PIB de EEUU (Billions) con Tendencia por Filtro de Hodrick-Prescott")
fig.add_scatter(x=gdp.index, y=tendencia['GDPC1'], mode='lines', name='Tendencia (HP)', line=dict(color='red', width=2))

In [54]:
tendencia

,GDPC1
0,2114.616270
1,2139.870604
2,2165.167478
3,2190.572572
4,2216.156104
...,...
309,23143.505005
310,23295.853596
311,23448.061010
312,23600.153234


In [55]:
gdp

,GDPC1
DATE,
1947-01-01,2182.681
1947-04-01,2176.892
1947-07-01,2172.432
1947-10-01,2206.452
1948-01-01,2239.682
...,...
2024-04-01,23223.906
2024-07-01,23400.294
2024-10-01,23542.349


In [56]:
ciclo = gdp - tendencia.values
ciclo

,GDPC1
DATE,
1947-01-01,68.064730
1947-04-01,37.021396
1947-07-01,7.264522
1947-10-01,15.879428
1948-01-01,23.525896
...,...
2024-04-01,80.400995
2024-07-01,104.440404
2024-10-01,94.287990


In [57]:
ciclo.plot(title="PIB de EEUU (Billions) con Componente Cíclico por Filtro de Hodrick-Prescott")

In [58]:
desviacion = 100 * (gdp / tendencia.values - 1)

In [59]:
desviacion

,GDPC1
DATE,
1947-01-01,3.218775
1947-04-01,1.730076
1947-07-01,0.335518
1947-10-01,0.724899
1948-01-01,1.061563
...,...
2024-04-01,0.347402
2024-07-01,0.448322
2024-10-01,0.402114


In [60]:
desviacion.plot(title='Desviacion del ciclo de tendencia')

Fuente de datos: <https://fred.stlouisfed.org/series/GDPC1/>

{{ termina_ejemplo }}





## El filtro HP tiene malas propiedades estadísticas
\cite{Hamilton:2017}: Why You Should Never Use the HP Filter?

1. El filtro Hodrick-Prescott introduce relaciones dinámicas espurias que no tienen sustento en el proceso generador de datos subyacente.
2. Los valores filtrados al final de la muestra son muy distintos de los del medio, y también están caracterizados por una dinámica espuria.
3. Una formalización estadística del problema típicamente produce valores de $\lambda$ que distan mucho de los usados comúnmente en la práctica.
4. Para Hamilton, hay una alternativa mejor: una regresión AR(4) alcanza todos los objetivos buscados por usuarios del filtro HP pero con ninguno de sus desventajas.



{{ empieza_ejemplo }} Filtrando el PIB de Costa Rica con HP {{ fin_titulo_ejemplo }}
Los datos filtrados son muy sensibles a nueva información.

```{figure} figures/pib-hp-tails.png

Ciclo del PIB de Costa Rica, conforme se van agregando nuevas observaciones
```
{{ termina_ejemplo }}



## HP puede inducir conclusiones equivocadas acerca del comovimiento de series
\textcite{CogleyNason:1995} analizaron las propiedades espectrales del filtro HP

Cuando se mide el componente cíclico de una serie de tiempo, ¿es buena idea usar el filtro HP?

Depende de la serie original

- **Sí**, si es estacionaria alrededor de tendencia
- **No**, si es estacionaria en diferencia

Este resultado tiene implicaciones importantes para modelos DSGE: Cuando se aplica el filtro HP a una serie integrada, el filtro introduce periodicidad y comovimiento en las frecuencias del ciclo económico, **aún si no estaban presentes en los datos originales**.